#OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction
In this [competition](https://https://www.kaggle.com/c/stanford-covid-vaccine/overview), you will be predicting the degradation rates at various locations along RNA sequence.

There are multiple ground truth values provided in the training data. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Library Required

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional,Embedding,Dropout, GRU
from tensorflow.keras.callbacks import ModelCheckpoint
import plotly.express as px


#The dataset
The mRNA vaccine degradation [dataset](https://https://www.kaggle.com/c/stanford-covid-vaccine/data) includes train and test.json files.
We will predict the degradation rate based on these three features:
['sequence', 'structure', 'predicted_loop_type']. All of these features are text (i.e. sequence of characters).

Also, the dataset includes two columns indicate the qulaity of observiations:['signal_to_noise', 'SN_filter']. We will filter the three features based on these two filters columns.

From the data description in Kaggle, the targets are:['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C'].

Now we know our features and our target, let us start process the data then build the model.

In [ ]:
#get comp data
train = pd.read_json('/kaggle/input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('/kaggle/input/stanford-covid-vaccine/test.json', lines=True)
test.head(2)

,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91


In [ ]:
# dataset columns
print(train.columns)

Index(['index', 'id', 'sequence', 'structure', 'predicted_loop_type',
       'signal_to_noise', 'SN_filter', 'seq_length', 'seq_scored',
       'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C'],
      dtype='object')


In [ ]:
# one of the target sequence length
len(train['reactivity'].values[0])

68

In [ ]:
# one of the features sequence length
len(train['sequence'].values[0])

107

In [ ]:
# check the filter columns
train['SN_filter'].value_counts()

1    1589
0     811
Name: SN_filter, dtype: int64

In [ ]:
# we can filter either with 'SN_filter'=1 or with 'signal_to_noise'>1
# filt = train['SN_filter']==1
# df = train.loc[filt]
# df.head(2)

In [ ]:
# df is our filtered train dataset
filt = train['signal_to_noise']>1
df = train.loc[filt]
df.head(2)

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."


In [ ]:
# these counters to check the sequences we have
# what characters they have?, how many for each one of them
from collections import Counter
RNA_c = Counter()
RNA_c2 = Counter()
RNA_c3 = Counter()

In [ ]:
u = [RNA_c.update(x) for i in range(len(df['sequence'])) for x in df['sequence'].values[i]]
u = [RNA_c2.update(x) for i in range(len(df['predicted_loop_type'])) for x in df['predicted_loop_type'].values[i]]
u = [RNA_c3.update(x) for i in range(len(df['structure'])) for x in df['structure'].values[i]]

In [ ]:
RNA_c.items()

dict_items([('G', 52500), ('A', 89265), ('C', 47449), ('U', 35058)])

In [ ]:
RNA_c2.items()

dict_items([('E', 70445), ('S', 99990), ('H', 26118), ('B', 2885), ('X', 7471), ('I', 11778), ('M', 5585)])

In [ ]:
RNA_c3.items()

dict_items([('.', 124282), ('(', 49995), (')', 49995)])

In [ ]:
# this dict to convert the characters to numbers so we can use it in the model 
token2int = {x:i for i, x in enumerate('().GACUESHBXIM')}

In [ ]:
token2int

{'(': 0,
 ')': 1,
 '.': 2,
 'G': 3,
 'A': 4,
 'C': 5,
 'U': 6,
 'E': 7,
 'S': 8,
 'H': 9,
 'B': 10,
 'X': 11,
 'I': 12,
 'M': 13}

In [ ]:
# this function to process a given sequence and it returns numpy array in the correct shape
# this function takes a seq_length as argument so we can determine how much length we would like to use in the training
def process_data(seqs, seg_length=68, inputt =True):
    print(len(seqs))
    print(len(seqs[0]))

  # check if it is input, if not then it is output
  # the input needs to be converted while the output not as the output is numbers
    if(inputt):
        l= [[token2int[x] for x in seqs[i][:seg_length]] for i in range(len(seqs))]
    else:
        l= [[x for x in seqs[i]] for i in range(len(seqs))]
  
    # to put the processed sequences with the correct shape (#samples, length of sequence, #features)
    l = np.array(l).reshape(len(l),len(l[0]),1)
    print(l.shape)
    return l


In [ ]:
# convert the three features to number with correct shape
seqdata = process_data(df['sequence'].values, 107)
loopdata = process_data(df['predicted_loop_type'].values, 107)
strucdata = process_data(df['structure'].values, 107)

2096
107
(2096, 107, 1)
2096
107
(2096, 107, 1)
2096
107
(2096, 107, 1)


In [ ]:
# combine the three features togather so we can use them as input to our model
input_data = np.concatenate((seqdata,loopdata,strucdata),axis=2)
input_data.shape

(2096, 107, 3)

In [ ]:
#target columns
target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

In [ ]:
# convert the targets to the correct format and shape
reactivity_data = process_data(df['reactivity'].values,68, inputt=False)
Mg_pH10_data = process_data(df['deg_Mg_pH10'].values,68, inputt=False)
pH10_data = process_data(df['deg_pH10'].values,68, inputt=False)
Mg_50C_data = process_data(df['deg_Mg_50C'].values, 68,inputt=False)
deg_50C_data = process_data(df['deg_50C'].values,68, inputt=False)


2096
68
(2096, 68, 1)
2096
68
(2096, 68, 1)
2096
68
(2096, 68, 1)
2096
68
(2096, 68, 1)
2096
68
(2096, 68, 1)


In [ ]:
# combine the targets togather so we can use them as output to the model
output_data = np.concatenate((reactivity_data,Mg_pH10_data,pH10_data,Mg_50C_data,deg_50C_data),axis=2)
output_data.shape

(2096, 68, 5)

#The model
I have tried many different models in this competition. It is worth to notice that our output length is 68 (degradation rates) and the input length is 107 characters, this means we have degradation rates for the first 68 characters of the features. Therefore, we need to truncate everything in the model output after the 68 timestep, so we can match the output 68 degradation rate to the output of the model.

Something else to notice, the test set has  two length of the features, the first is 107 as same as the train set and this called public test set. The second is private testset and it has 130 features length, so we need to train on 107 and before the test we rebuild the model for the public and private test set length and load the weights.

In [ ]:
# def build_mode(inputLength, pred_length):
#     inputs = tf.keras.Input(shape=(inputLength,3))
#     lstm_output = Bidirectional(LSTM(64,return_sequences=True))(inputs)
#     lstm_output = lstm_output[:,:pred_length]
#     dense_output = Dense(5, activation='linear')(lstm_output)
#     model = tf.keras.Model(inputs, dense_output)
#     return model

In [ ]:
def build_mode(inputLength, pred_length):
    inputs = tf.keras.Input(shape=(inputLength,3))
    embd = Embedding(input_dim=14,output_dim=100)(inputs)
    embd = tf.reshape(embd, shape=(-1,embd.shape[1], embd.shape[2]*embd.shape[3]))
    lstm_output = Bidirectional(LSTM(64,return_sequences=True))(embd)
    lstm_output = lstm_output[:,:pred_length]
    dense_output = Dense(5, activation='linear')(lstm_output)
    model = tf.keras.Model(inputs, dense_output)
    return model


In [ ]:
# def build_mode(inputLength, pred_length):
#     inputs = tf.keras.Input(shape=(inputLength,3))
#     embd = Embedding(input_dim=30,output_dim=100)(inputs)
#     embd = tf.reshape(embd, shape=(-1,embd.shape[1], embd.shape[2]*embd.shape[3]))
#     lstm1_output = Bidirectional(LSTM(128,return_sequences=True))(embd)
#     lstm2_output = Bidirectional(LSTM(128,return_sequences=True))(lstm1_output)
#     lstm_output = lstm2_output[:,:pred_length]
#     dense_output = Dense(5, activation='linear')(lstm_output)
#     model = tf.keras.Model(inputs, dense_output)
#     return model


In [ ]:
# build model takes input 68 characters and predict 68 degradation rates
# model = build_mode(68, 68)

# build model takes input 107 characters and predict 68 degradation rates
model = build_mode(107, 68)

model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 107, 3)]          0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 107, 3, 100)       1400      
_________________________________________________________________
tf_op_layer_Reshape_1 (Tenso [(None, 107, 300)]        0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 107, 128)          186880    
_________________________________________________________________
tf_op_layer_strided_slice_1  [(None, 68, 128)]         0         
_________________________________________________________________
dense_1 (Dense)              (None, 68, 5)             645       
Total params: 188,925
Trainable params: 188,925
Non-trainable params: 0
________________________________________________

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['acc'])

In [ ]:
# to return the best weights of the model with less loss values.
def mcp_save():
  filepath= "weights.best.hdf5"
  return ModelCheckpoint(filepath, save_best_only=True, monitor='loss', mode='min', verbose=0)

In [ ]:
# to choose the hyperparameter before the actual training and we use valdation set here
# history = model.fit(input_data,output_data, epochs=200,validation_split=0.3,callbacks=[mcp_save()])
# the actual training with the whole train data
history = model.fit(input_data,output_data, epochs=80,callbacks=[mcp_save()])

Epoch 1/80
66/66 [==============================] - 1s 15ms/step - loss: 0.2186 - acc: 0.3228
Epoch 2/80
66/66 [==============================] - 1s 14ms/step - loss: 0.1530 - acc: 0.3692
Epoch 3/80
66/66 [==============================] - 1s 14ms/step - loss: 0.1340 - acc: 0.3891
Epoch 4/80
66/66 [==============================] - 1s 16ms/step - loss: 0.1258 - acc: 0.3961
Epoch 5/80
66/66 [==============================] - 1s 16ms/step - loss: 0.1207 - acc: 0.4041
Epoch 6/80
66/66 [==============================] - 1s 14ms/step - loss: 0.1157 - acc: 0.4070
Epoch 7/80
66/66 [==============================] - 1s 15ms/step - loss: 0.1119 - acc: 0.4116
Epoch 8/80
66/66 [==============================] - 1s 14ms/step - loss: 0.1082 - acc: 0.4104
Epoch 9/80
66/66 [==============================] - 1s 14ms/step - loss: 0.1060 - acc: 0.4107
Epoch 10/80
66/66 [==============================] - 1s 14ms/step - loss: 0.1039 - acc: 0.4115
Epoch 11/80
66/66 [==============================] - 1s 14m

In [ ]:
# Visualize the val-loss and training loss
fig = px.line(
    history.history, y=['loss', 'val_loss'], 
    labels={'index': 'epoch', 'value': 'Mean Squared Error'}, 
    title='Training History')
fig.show()

In [ ]:
# take both lengths of the test set
publich_test = test.query('seq_length ==107')
private_test = test.query('seq_length ==130')

In [ ]:
# convert the features with length 107 from the testset and combine them
seqdata_pubtest = process_data(publich_test['sequence'].values,107)
loopdata_pubtest = process_data(publich_test['predicted_loop_type'].values,107)
strucdata_pubtest = process_data(publich_test['structure'].values,107)
input_pubtest_data = np.concatenate((seqdata_pubtest,loopdata_pubtest,strucdata_pubtest),axis=2)
input_pubtest_data.shape

629
107
(629, 107, 1)
629
107
(629, 107, 1)
629
107
(629, 107, 1)


(629, 107, 3)

In [ ]:
# convert the features with length 130 from the testset and combine them
seqdata_pritest = process_data(private_test['sequence'].values,130)
loopdata_pritest = process_data(private_test['predicted_loop_type'].values,130)
strucdata_pritest = process_data(private_test['structure'].values,130)
input_pritest_data = np.concatenate((seqdata_pritest,loopdata_pritest,strucdata_pritest),axis=2)
input_pritest_data.shape

3005
130
(3005, 130, 1)
3005
130
(3005, 130, 1)
3005
130
(3005, 130, 1)


(3005, 130, 3)

In [ ]:
# rebuild the models with both lengths
model_public = build_mode(107, 107)
model_private = build_mode(130, 130)
# load the weights
model_public.load_weights('weights.best.hdf5')
model_private.load_weights('weights.best.hdf5')

In [ ]:
# make predictions on the testset
public_preds = model_public.predict(input_pubtest_data)
private_preds = model_private.predict(input_pritest_data)

In [ ]:
# This will tell us the columns we are predicting
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

In [ ]:
# preapre the submission file to be in the correct format (a prediction per base)
preds_ls = []

for df, preds in [(publich_test, public_preds), (private_test, private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)
preds_df.head()

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C,id_seqpos
0,0.580043,0.659048,2.106254,0.554200,0.766238,id_00073f8be_0
1,1.878288,2.774827,3.849584,2.857925,2.586583,id_00073f8be_1
2,1.106511,0.452194,0.553056,0.699635,0.731798,id_00073f8be_2
3,1.256432,1.080847,1.151894,1.534393,1.470501,id_00073f8be_3
4,0.828744,0.597933,0.527194,0.854053,0.780136,id_00073f8be_4


In [ ]:
data_dir = '/kaggle/input/stanford-covid-vaccine/'
sample_df = pd.read_csv(data_dir + 'sample_submission.csv')

In [ ]:
preds_df.shape

In [ ]:
# save the submission file
submission = sample_df[['id_seqpos']].merge(preds_df, on=['id_seqpos'])
submission.to_csv('submission1.csv', index=False)

# References
https://www.kaggle.com/xhlulu/openvaccine-simple-gru-model

https://www.kaggle.com/tuckerarrants/openvaccine-gru-lstm#Training
